In [1]:
#LSTM 사용하여 버튼 하나의 x좌표 예측

import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [40]:
np.random.seed(7) #결과를 재현할 수 있도록 난수 시드 수정
data1=pd.read_excel('x_data.xls') #x좌표 엑셀 데이터 불러옴 
data2=pd.read_excel('y_data.xls') #x좌표 엑셀 데이터 불러옴 
data3=pd.read_excel('speed.xls') #x좌표 엑셀 데이터 불러옴 

pddata1=pd.DataFrame(data1) 
pddata1.head()
pddata2=pd.DataFrame(data2) 
pddata2.head()
pddata3=pd.DataFrame(data3) 
pddata3.head()

hap=[] #진짜 합
for j in range(len(pddata1)):
    x_data=np.array(pddata1.loc[j])
    y_data=np.array(pddata2.loc[j])
    s_data=np.array(pddata3.loc[j])

    hap1=[] #[x,y,속력] 데이터를 저장하고 있는 리스트

    for i in range(len(s_data)): 
        sum=[] #리스트 하나당 임시로 x,y,속력을 저장할 리스트 (for문 돌릴때마다 초기화)
        sum.append(x_data[i]) #x넣음
        sum.append(y_data[i]) #y넣음
        sum.append(s_data[i]) #속력넣음
        hap1.append(sum) #[x,y,속력] 하나의 리스트를 hap리스트에 넣음  

    j = j + 1
    
    hap.append(hap1)

n1 = np.array(hap)
print(n1.shape) #257,99,3 257개의 샘플과 99개 시계열, 3개 피쳐(x,y,속력)


(257, 99, 3)


In [41]:
num_sample = n1.shape[0] # 257개 샘플 데이터
num_sequence = n1.shape[1] # 99개 시계열 데이터
num_feature = n1.shape[2] #3개 피쳐

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() #클래스의 인스턴스 생성


#시계열 선회하면서 피팅함
for ss in range(num_sequence):
    scaler.partial_fit(n1[:,ss,:])
    
#Numpy 활용하여 3차원 데이터 스케일링(변환) 0~1 스케일링
results=[]
for ss in range(num_sequence):
    results.append(scaler.transform(n1[:,ss,:]).reshape(num_sample,1,num_feature))
n1_scaled = np.concatenate(results, axis=1)

dataset=  n1_scaled

In [42]:
dataset

array([[[0.13551666, 0.08466454, 0.        ],
        [0.13551666, 0.08466454, 0.        ],
        [0.13551666, 0.08466454, 0.        ],
        ...,
        [0.04291361, 0.25239617, 0.        ],
        [0.04291361, 0.25239617, 0.        ],
        [0.04291361, 0.25239617, 0.        ]],

       [[0.03500847, 0.27476038, 0.        ],
        [0.03500847, 0.27476038, 0.        ],
        [0.03500847, 0.27476038, 0.        ],
        ...,
        [0.10559006, 0.36261981, 0.        ],
        [0.10559006, 0.36261981, 0.        ],
        [0.10559006, 0.36261981, 0.01649124]],

       [[0.23433089, 0.20607029, 0.        ],
        [0.23433089, 0.20607029, 0.        ],
        [0.23433089, 0.20607029, 0.        ],
        ...,
        [0.14624506, 0.37380192, 0.        ],
        [0.14624506, 0.37380192, 0.        ],
        [0.14624506, 0.37380192, 0.        ]],

       ...,

       [[0.14511575, 0.12140575, 0.        ],
        [0.14511575, 0.12140575, 0.        ],
        [0.14511575, 0

In [43]:
train_size = int(len(dataset) * 0.67) #학습 데이터 67%
test_size = len(dataset) - train_size #테스트 데이터 33%
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

# look_back = 1 #이전 시간 단계 입력변수
# #X는 지금 t 값이고, Y는 그 다음의 t+1 값임 (즉, X=t, Y=t+1)
# trainX, trainY = create_dataset(train, look_back)
# testX, testY = create_dataset(test, look_back)
# LSTM 모델은 입력데이터 X에 [samples, time steps, features]형식으로 넣어야함
# 현재 데이터는 [samples, features]이므로, numpy.reshape()를 사용하여
#준비된 train과 test 입력 데이터를 3차원으로 바꿈
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))



# # dataframe=pd.DataFrame(hap)
# dataframe=pd.DataFrame(data1) #DataFrame은 2차원 테이블 데이터 구조 자료형

# dataset = dataframe.values #데이터 프레임에서 numpy배열 추출

# dataset = dataset.astype('float32') #정수값을 부동 소수점 값으로 변환
# # LSTM 은 S자형(기본값) 또는 tanh활성화 기능을 사용할 떄 입력 데이터의 스케일에 민감
# scaler = MinMaxScaler(feature_range=(0, 1)) #정규화함 (0에서 1까지의 범위로 데이터 크기 조정)
# dataset = scaler.fit_transform(dataset.reshape(-1,1))
#reshape함수를 사용하여 1차원 데이터를 2차원으로 변경

# dataset=  n1_scaled

# train_size = int(len(dataset) * 0.67) #학습 데이터 67%
# test_size = len(dataset) - train_size #테스트 데이터 33%
# train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

#두개의 인수, 데이터 세트로 변환하려는 Numpt 배열인 dataset과 다음 시간을 예측하기
#위해 입력 변수로 사용할 이전 시간 단계 수인 look_back을 사용. 기본값을 1

# look_back = 1 #이전 시간 단계 입력변수
# #X는 지금 t 값이고, Y는 그 다음의 t+1 값임 (즉, X=t, Y=t+1)
# trainX, trainY = create_dataset(train, look_back)
# testX, testY = create_dataset(test, look_back)
# # LSTM 모델은 입력데이터 X에 [samples, time steps, features]형식으로 넣어야함
# # 현재 데이터는 [samples, features]이므로, numpy.reshape()를 사용하여
# #준비된 train과 test 입력 데이터를 3차원으로 바꿈
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [44]:
train

array([[[0.13551666, 0.08466454, 0.        ],
        [0.13551666, 0.08466454, 0.        ],
        [0.13551666, 0.08466454, 0.        ],
        ...,
        [0.04291361, 0.25239617, 0.        ],
        [0.04291361, 0.25239617, 0.        ],
        [0.04291361, 0.25239617, 0.        ]],

       [[0.03500847, 0.27476038, 0.        ],
        [0.03500847, 0.27476038, 0.        ],
        [0.03500847, 0.27476038, 0.        ],
        ...,
        [0.10559006, 0.36261981, 0.        ],
        [0.10559006, 0.36261981, 0.        ],
        [0.10559006, 0.36261981, 0.01649124]],

       [[0.23433089, 0.20607029, 0.        ],
        [0.23433089, 0.20607029, 0.        ],
        [0.23433089, 0.20607029, 0.        ],
        ...,
        [0.14624506, 0.37380192, 0.        ],
        [0.14624506, 0.37380192, 0.        ],
        [0.14624506, 0.37380192, 0.        ]],

       ...,

       [[0.10333145, 0.63258786, 0.        ],
        [0.10333145, 0.63258786, 0.        ],
        [0.10333145, 0

In [45]:
test

array([[[0.10446076, 0.08466454, 0.        ],
        [0.10446076, 0.08466454, 0.        ],
        [0.10446076, 0.08466454, 0.        ],
        ...,
        [0.19649915, 0.28913738, 0.08489383],
        [0.20158103, 0.2971246 , 0.        ],
        [0.20158103, 0.2971246 , 0.07093148]],

       [[0.24957651, 0.45047923, 0.        ],
        [0.24957651, 0.45047923, 0.        ],
        [0.24957651, 0.45047923, 0.        ],
        ...,
        [0.12704687, 0.36102236, 0.05214986],
        [0.12365895, 0.35782748, 0.        ],
        [0.12365895, 0.35782748, 0.02332213]],

       [[0.12027103, 0.34824281, 0.        ],
        [0.12027103, 0.34824281, 0.        ],
        [0.12027103, 0.34824281, 0.        ],
        ...,
        [0.0920384 , 0.26996805, 0.        ],
        [0.0920384 , 0.26996805, 0.077789  ],
        [0.08752117, 0.26198083, 0.        ]],

       ...,

       [[0.14511575, 0.12140575, 0.        ],
        [0.14511575, 0.12140575, 0.        ],
        [0.14511575, 0

In [46]:
np.shape(train)

(172, 99, 3)

In [47]:
np.shape(test)

(85, 99, 3)

In [48]:
len(dataset)

257

In [89]:
#배열을 행렬로 변한
def create_dataset(dataset, look_back=1):
    dataX, dataY = [],[] 
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back):]
        dataX.append(a)
        dataY.append(dataset[i:(i + look_back):])
    
    dataX=np.squeeze(dataX,axis=1) #차원 축소
    dataY=np.squeeze(dataY,axis=1) #차원 축소
    return np.array(dataX), np.array(dataY)

In [90]:
#두개의 인수, 데이터 세트로 변환하려는 Numpy 배열인 dataset과 다음 시간을 예측하기
#위해 입력 변수로 사용할 이전 시간 단계 수인 look_back을 사용. 기본값을 1
#reshpae()함수를 사용하여 배열의 차원을 변경할 수 있음

look_back = 1 #이전 시간 단계 입력변수
#X는 지금 t 값이고, Y는 그 다음의 t+1 값임 (즉, X=t, Y=t+1)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# LSTM 모델은 입력데이터 X에 [samples, time steps, features]형식으로 넣어야함
# 현재 데이터는 [samples, features]이므로, numpy.reshape()를 사용하여
#준비된 train과 test 입력 데이터를 3차원으로 바꿈
# trainX = np.reshape(trainX.shape[0], trainX.shape[1], trainX.shape[2])
# testX = np.reshape(testX.shape[0], testX.shape[1], testX.shape[2])

In [91]:
np.shape(trainX)

(170, 99, 3)

In [92]:
np.shape(testX)

(83, 99, 3)

In [93]:
train

array([[[0.13551666, 0.08466454, 0.        ],
        [0.13551666, 0.08466454, 0.        ],
        [0.13551666, 0.08466454, 0.        ],
        ...,
        [0.04291361, 0.25239617, 0.        ],
        [0.04291361, 0.25239617, 0.        ],
        [0.04291361, 0.25239617, 0.        ]],

       [[0.03500847, 0.27476038, 0.        ],
        [0.03500847, 0.27476038, 0.        ],
        [0.03500847, 0.27476038, 0.        ],
        ...,
        [0.10559006, 0.36261981, 0.        ],
        [0.10559006, 0.36261981, 0.        ],
        [0.10559006, 0.36261981, 0.01649124]],

       [[0.23433089, 0.20607029, 0.        ],
        [0.23433089, 0.20607029, 0.        ],
        [0.23433089, 0.20607029, 0.        ],
        ...,
        [0.14624506, 0.37380192, 0.        ],
        [0.14624506, 0.37380192, 0.        ],
        [0.14624506, 0.37380192, 0.        ]],

       ...,

       [[0.10333145, 0.63258786, 0.        ],
        [0.10333145, 0.63258786, 0.        ],
        [0.10333145, 0

In [114]:
trainY

array([[[0.13551666, 0.08466454, 0.        ],
        [0.13551666, 0.08466454, 0.        ],
        [0.13551666, 0.08466454, 0.        ],
        ...,
        [0.04291361, 0.25239617, 0.        ],
        [0.04291361, 0.25239617, 0.        ],
        [0.04291361, 0.25239617, 0.        ]],

       [[0.03500847, 0.27476038, 0.        ],
        [0.03500847, 0.27476038, 0.        ],
        [0.03500847, 0.27476038, 0.        ],
        ...,
        [0.10559006, 0.36261981, 0.        ],
        [0.10559006, 0.36261981, 0.        ],
        [0.10559006, 0.36261981, 0.01649124]],

       [[0.23433089, 0.20607029, 0.        ],
        [0.23433089, 0.20607029, 0.        ],
        [0.23433089, 0.20607029, 0.        ],
        ...,
        [0.14624506, 0.37380192, 0.        ],
        [0.14624506, 0.37380192, 0.        ],
        [0.14624506, 0.37380192, 0.        ]],

       ...,

       [[0.23094297, 0.19329073, 0.        ],
        [0.23094297, 0.19329073, 0.        ],
        [0.23094297, 0

In [150]:
#LSTM 모델 생성

from keras.layers import TimeDistributed
from keras.layers import InputLayer
from keras.layers import Reshape

model = Sequential() # Sequeatial Model 

model.add(LSTM(32, return_sequences=True, input_shape=trainX.shape[-2:]))  
# 20: 메모리 셀의 개수 input_shape: (timestep, feature) 

model.add(Dense(3)) # 3개의 예측

model.compile(loss='mean_squared_error', optimizer='adam') 
model.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 99, 32)            4608      
_________________________________________________________________
dense_19 (Dense)             (None, 99, 3)             99        
Total params: 4,707
Trainable params: 4,707
Non-trainable params: 0
_________________________________________________________________


In [151]:
model.fit(trainX, trainY, epochs=5,batch_size=30, verbose=1)

Epoch 1/5
170/170 [==============================] - 1s 6ms/step - loss: 0.0717
Epoch 2/5
170/170 [==============================] - 0s 2ms/step - loss: 0.0424
Epoch 3/5
170/170 [==============================] - 0s 2ms/step - loss: 0.0204
Epoch 4/5
170/170 [==============================] - 0s 2ms/step - loss: 0.0090
Epoch 5/5
170/170 [==============================] - 0s 3ms/step - loss: 0.0079


In [153]:
# 예측

# nsamples, nx, ny=trainX.shape
# trainX=trainX.reshape((nsamples,nx*ny))
# nsamples, nx, ny=testX.shape
# testX=testX.reshape((nsamples,nx*ny))

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

#inverse_trasform()함수를 사용하여 스케일링된 결과 값을 본래 값으로 구함
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY]) 
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
#원래 데이터셋 : 파란색, 테스트 데이터셋 : 녹색 #훈련 데이터셋 : 주황색
#train 예측 
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

plt.plot(scaler.inverse_transform(dataset)) #전체 데이터
plt.show()
plt.plot(trainPredictPlot) #초반 부분 train 데이터
plt.show()
plt.plot(testPredictPlot) #후반 부분 test 데이터
plt.show()

plt.plot(scaler.inverse_transform(dataset)) #전체 데이터
plt.plot(trainPredictPlot) #초반 부분 train 데이터
plt.plot(testPredictPlot) #후반 부분 test 데이터
plt.show()